# Haber Tahmini İçin En Uygun Makine Öğrenmesi Algoritmasının Bulunması

In [1]:
from textblob import TextBlob
from sklearn import model_selection, preprocessing ,linear_model,naive_bayes, metrics
from sklearn.feature_extraction.text import TfidfVectorizer ,CountVectorizer
from sklearn import decomposition, ensemble

import pandas ,xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

Using TensorFlow backend.
D:\Anakonda\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Anakonda\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Anakonda\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Anakonda\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as

In [2]:
import pandas as pd #Verileri okumak
data1 = pd.read_csv('onedioÇokAcı.csv')
data2 = pd.read_csv('onedioKızgın2.csv')
data3 = pd.read_csv('thegeyikHayvanlar.csv')
data4 = pd.read_csv('thegeyikKomik.csv')
data5 = pd.read_csv('thegeyikSeyehat.csv')
data6 = pd.read_csv('onedioGoygoy.csv')
data7 = pd.read_csv('onedioGoygoy2.csv')
data=pd.concat([data1,data2,data3,data4,data5,data6,data7], ignore_index = True) #Verileri birleştirmek
data

,Text,Sentiment
0,Evi Terk Eden Eşi Sevgililer Günü’nde Barışmay...,0
1,Yeliz Koray’ın 6 Yıldır Terör Örgütünün Elinde...,0
2,"Geçim Sıkıntısı Çeken Anne ve Baba, Çocukların...",0
3,Türk Denizciyle Hırvat Sevgilisinin Filmlere T...,0
4,İtalya'daki Türk Öğrencinin Bursuyla Yaptığı M...,0
...,...,...
17203,Kitaplara Göre Yapılmış 17 Karakter Analizi,1
17204,Final Sınavlarına Mizahıyla İsyan Eden 18 Kişi,1
17205,Alakasız Sohbetler Kanalından Eğlendirirken Öğ...,1
17206,"Bir Uçakta Yaşanmış ""Oldu Olacak Deve Getirsey...",1


## Sentiment Analizi Ve Sınıflandırma Modelleri

In [3]:
data["Sentiment"].replace(0,value = "negatif" ,inplace = True) #0 lara negatif 
data["Sentiment"].replace(1,value = "pozitif" ,inplace = True) #1 lere pozitif

In [4]:
data

,Text,Sentiment
0,Evi Terk Eden Eşi Sevgililer Günü’nde Barışmay...,negatif
1,Yeliz Koray’ın 6 Yıldır Terör Örgütünün Elinde...,negatif
2,"Geçim Sıkıntısı Çeken Anne ve Baba, Çocukların...",negatif
3,Türk Denizciyle Hırvat Sevgilisinin Filmlere T...,negatif
4,İtalya'daki Türk Öğrencinin Bursuyla Yaptığı M...,negatif
...,...,...
17203,Kitaplara Göre Yapılmış 17 Karakter Analizi,pozitif
17204,Final Sınavlarına Mizahıyla İsyan Eden 18 Kişi,pozitif
17205,Alakasız Sohbetler Kanalından Eğlendirirken Öğ...,pozitif
17206,"Bir Uçakta Yaşanmış ""Oldu Olacak Deve Getirsey...",pozitif


In [5]:
data.groupby("Sentiment").count()

,Text
Sentiment,
negatif,5694
pozitif,11514


In [6]:
df = pd.DataFrame() 
df["text"] = data["Text"]
df["label"] = data ["Sentiment"]
df.head()

,text,label
0,Evi Terk Eden Eşi Sevgililer Günü’nde Barışmay...,negatif
1,Yeliz Koray’ın 6 Yıldır Terör Örgütünün Elinde...,negatif
2,"Geçim Sıkıntısı Çeken Anne ve Baba, Çocukların...",negatif
3,Türk Denizciyle Hırvat Sevgilisinin Filmlere T...,negatif
4,İtalya'daki Türk Öğrencinin Bursuyla Yaptığı M...,negatif


## Metin Ön İşleme

In [7]:
#buyuk-kucuk donusumu
df['text'] = df['text'].apply(lambda x: " ".join(x.lower() for x in x.split()))
#noktalama işaretleri
df['text'] = df['text'].str.replace('[^\w\s]','')
#sayılar
df['text'] = df['text'].str.replace('\d','')

In [8]:
pd.Series(" ".join(df["text"]).split()).value_counts() 

bir           2627
ve            1873
en            1822
komik         1821
kişi          1737
              ... 
kahvesini        1
ödesek           1
yaşatarak        1
getirtecek       1
gelmesini        1
Length: 35623, dtype: int64

In [9]:
#stopwords
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
sw = stopwords.words('Turkish')
df['text'] = df['text'].apply(lambda x: " ".join(x for x in x.split() if x not in sw))

In [10]:
print(sw) #Stopwordsler aslında dildeki sık kullanılan kelimeler bunları veri setimizde istemiyoruz.

['acaba', 'ama', 'aslında', 'az', 'bazı', 'belki', 'biri', 'birkaç', 'birşey', 'biz', 'bu', 'çok', 'çünkü', 'da', 'daha', 'de', 'defa', 'diye', 'eğer', 'en', 'gibi', 'hem', 'hep', 'hepsi', 'her', 'hiç', 'için', 'ile', 'ise', 'kez', 'ki', 'kim', 'mı', 'mu', 'mü', 'nasıl', 'ne', 'neden', 'nerde', 'nerede', 'nereye', 'niçin', 'niye', 'o', 'sanki', 'şey', 'siz', 'şu', 'tüm', 've', 'veya', 'ya', 'yani']


In [11]:
# Burada kelime köklerini 
from snowballstemmer import TurkishStemmer
turkStem=TurkishStemmer()
#nltk.download('wordnet')
from textblob import Word
 #nltk.download('wordnet')
df['text'] = df['text'].apply(lambda x: " ".join([turkStem.stemWord(word) for word in x.split()])) #kelimelerin köklerine indirgedik. Bu işlem veri işleme için önemli bir adım.

In [12]:
#seyreklerin silinmesi
sil = pd.Series(' '.join(df['text']).split()).value_counts()[-1000:]
df['text'] = df['text'].apply(lambda x: " ".join(x for x in x.split() if x not in sil))

In [13]:
df.to_csv("ÖnİşlenmişVeriler2.csv",index = False)

## Değişken Mühendisliği

Bir şekilde metinleri ölcülebilir metriklere dönüştürmek istiyoruz. 
- Count Vectors: Basit sayma işlemleri
- TF-IDF Vectors(words,chracters,n-grams)
- TF(t) = (Bir t teriminin bir dökümanda gözlenme frekansı / dökümandaki toplam terim sayısı)
- IDF(t) = log_e(Toplam döküman sayısı içinde t terimi olan belge sayısı)

## Test-Train

In [14]:
train_x ,test_x,train_y,test_y = model_selection.train_test_split(df["text"],
                                                                 df["label"])

In [15]:
encoder = preprocessing.LabelEncoder()

In [16]:
train_y = encoder.fit_transform(train_y)
test_y = encoder.fit_transform(test_y)

## Count Vector

In [17]:
vectorizer = CountVectorizer()
vectorizer.fit(train_x)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [18]:
X_train_count = vectorizer.transform(train_x)
X_test_count = vectorizer.transform(test_x)

In [19]:
vectorizer.get_feature_names()[10:20] 

['abartmama',
 'abartmış',
 'abarttık',
 'abartı',
 'abartıl',
 'abartılır',
 'abartılıyor',
 'abartıp',
 'abazoğl',
 'abbasoğlu']

In [20]:
X_train_count.toarray() 

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

## TDF-IDF(words)

In [21]:
tf_idf_word_vectorizer = TfidfVectorizer()
tf_idf_word_vectorizer.fit(train_x)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [22]:
X_train_tf_idf_word = tf_idf_word_vectorizer.transform(train_x)
X_test_tf_idf_word = tf_idf_word_vectorizer.transform(test_x)

In [23]:
tf_idf_word_vectorizer.get_feature_names()[10:50] #içerisindeki verilere bakmak.

['abartmama',
 'abartmış',
 'abarttık',
 'abartı',
 'abartıl',
 'abartılır',
 'abartılıyor',
 'abartıp',
 'abazoğl',
 'abbasoğlu',
 'abd',
 'abdde',
 'abddek',
 'abde',
 'abdli',
 'abdn',
 'abdulhamidi',
 'abdulkadir',
 'abdullah',
 'abdurrahma',
 'abdye',
 'abdülhamidi',
 'abdülhamit',
 'abdülmeci',
 'abi',
 'abides',
 'abidi',
 'abidik',
 'abigail',
 'abilerdenabla',
 'abis',
 'abla',
 'ablas',
 'ablukas',
 'abonelik',
 'absürdü',
 'absürt',
 'absürtlük',
 'abu',
 'abuk']

In [24]:
X_train_tf_idf_word.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

## TDF-IDF(ngram)

In [25]:
tf_idf_ngram_vectorizer = TfidfVectorizer(ngram_range=(2,3))
tf_idf_ngram_vectorizer.fit(train_x)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(2, 3), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [26]:
X_train_tf_idf_ngram = tf_idf_ngram_vectorizer.transform(train_x)
X_test_tf_idf_ngram = tf_idf_ngram_vectorizer.transform(test_x)

## TDF-IDF(Characters)

In [27]:
tf_idf_chars_vectorizer = TfidfVectorizer(analyzer = "char", ngram_range=(2,3))
tf_idf_chars_vectorizer.fit(train_x)

TfidfVectorizer(analyzer='char', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(2, 3), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [28]:
X_train_tf_idf_chars = tf_idf_chars_vectorizer.transform(train_x)
X_test_tf_idf_chars = tf_idf_chars_vectorizer.transform(test_x)

## Makine Öğrenmesi İle Doğruluk Oranlarının Bulunması

## Logistic Regression

In [46]:
loj = linear_model.LogisticRegression()
loj_model = loj.fit(X_train_count, train_y)
accuracy = model_selection.cross_val_score(loj_model, 
                                           X_test_count, 
                                           test_y, 
                                           cv = 10).mean()

print("Count Vectors Doğruluk Oranı:", accuracy)

Count Vectors Doğruluk Oranı: 0.8856304969513842


In [30]:
loj = linear_model.LogisticRegression()
loj_model = loj.fit(X_train_tf_idf_word,train_y)
accuracy = model_selection.cross_val_score(loj_model, 
                                           X_test_tf_idf_word, 
                                           test_y, 
                                           cv = 10).mean()

print("TF-IDF Doğruluk Oranı:", accuracy)

TF-IDF Doğruluk Oranı: 0.8670420331300923


In [31]:
loj = linear_model.LogisticRegression()
loj_model = loj.fit(X_train_tf_idf_ngram,train_y)
accuracy = model_selection.cross_val_score(loj_model, 
                                           X_test_tf_idf_ngram, 
                                           test_y, 
                                           cv = 10).mean()

print("N-GRAM TF-IDF Doğruluk Oranı:", accuracy)

N-GRAM TF-IDF Doğruluk Oranı: 0.7287341498947822


In [32]:
loj = linear_model.LogisticRegression()
loj_model = loj.fit(X_train_tf_idf_chars,train_y)
accuracy = model_selection.cross_val_score(loj_model, 
                                           X_test_tf_idf_chars, 
                                           test_y, 
                                           cv = 10).mean()

print("CHARLEVEL Doğruluk Oranı:", accuracy)

CHARLEVEL Doğruluk Oranı: 0.86169157718664


## Naive Bayes

In [34]:
nb = naive_bayes.MultinomialNB()
nb_model = nb.fit(X_train_tf_idf_word,train_y)
accuracy = model_selection.cross_val_score(nb_model, 
                                           X_test_tf_idf_word, 
                                           test_y, 
                                           cv = 10).mean()

print("Word-Level TF-IDF Doğruluk Oranı:", accuracy)

Word-Level TF-IDF Doğruluk Oranı: 0.8175298116872606


In [35]:
nb = naive_bayes.MultinomialNB()
nb_model = nb.fit(X_train_tf_idf_ngram,train_y)
accuracy = model_selection.cross_val_score(nb_model, 
                                           X_test_tf_idf_ngram, 
                                           test_y, 
                                           cv = 10).mean()

print("N-GRAM TF-IDF Doğruluk Oranı:", accuracy)

N-GRAM TF-IDF Doğruluk Oranı: 0.7350089030378244


In [36]:
nb = naive_bayes.MultinomialNB()
nb_model = nb.fit(X_train_tf_idf_chars,train_y)
accuracy = model_selection.cross_val_score(nb_model, 
                                           X_test_tf_idf_chars, 
                                           test_y, 
                                           cv = 10).mean()

print("CHARLEVEL Doğruluk Oranı:", accuracy)

CHARLEVEL Doğruluk Oranı: 0.7870814223277397


## Random Forest 

In [37]:
rf = ensemble.RandomForestClassifier()
rf_model = rf.fit(X_train_count,train_y)
accuracy = model_selection.cross_val_score(rf_model, 
                                           X_test_count, 
                                           test_y, 
                                           cv = 10).mean()

print("Count Vectors Doğruluk Oranı:", accuracy)

Count Vectors Doğruluk Oranı: 0.8623876328710949


In [38]:
rf = ensemble.RandomForestClassifier()
rf_model = rf.fit(X_train_tf_idf_word,train_y)
accuracy = model_selection.cross_val_score(rf_model, 
                                           X_test_tf_idf_word, 
                                           test_y, 
                                           cv = 10).mean()

print("Word-Level TF-IDF Doğruluk Oranı:", accuracy)

Word-Level TF-IDF Doğruluk Oranı: 0.8586710192629363


In [39]:
rf = ensemble.RandomForestClassifier()
rf_model = loj.fit(X_train_tf_idf_ngram,train_y)
accuracy = model_selection.cross_val_score(rf_model, 
                                           X_test_tf_idf_ngram, 
                                           test_y, 
                                           cv = 10).mean()

print("N-GRAM TF-IDF Doğruluk Oranı:", accuracy)

N-GRAM TF-IDF Doğruluk Oranı: 0.7287341498947822


In [40]:
rf = ensemble.RandomForestClassifier()
rf_model = loj.fit(X_train_tf_idf_chars,train_y)
accuracy = model_selection.cross_val_score(rf_model, 
                                           X_test_tf_idf_chars, 
                                           test_y, 
                                           cv = 10).mean()

print("CHARLEVEL Doğruluk Oranı:", accuracy)

CHARLEVEL Doğruluk Oranı: 0.86169157718664


## XGboost

In [41]:
xgb = xgboost.XGBClassifier()
xgb_model = xgb.fit(X_train_count,train_y)
accuracy = model_selection.cross_val_score(xgb_model, 
                                           X_test_count, 
                                           test_y, 
                                           cv = 10).mean()

print("Count Vectors Doğruluk Oranı:", accuracy)

Count Vectors Doğruluk Oranı: 0.7821923056170075


In [42]:
xgb = xgboost.XGBClassifier()
xgb_model = xgb.fit(X_train_tf_idf_word,train_y)
accuracy = model_selection.cross_val_score(xgb_model, 
                                           X_test_tf_idf_word, 
                                           test_y, 
                                           cv = 10).mean()

print("Word-Level TF-IDF Doğruluk Oranı:", accuracy)

Word-Level TF-IDF Doğruluk Oranı: 0.778706091836184


In [43]:
xgb = xgboost.XGBClassifier()
xgb_model = xgb.fit(X_train_tf_idf_chars,train_y)
accuracy = model_selection.cross_val_score(xgb_model, 
                                           X_test_tf_idf_chars, 
                                           test_y, 
                                           cv = 10).mean()

print("CHARLEVEL Doğruluk Oranı:", accuracy)

CHARLEVEL Doğruluk Oranı: 0.8207888631090487


### En iyi Doğruluk oranını veren 0.8856304969513842 doğruluk oranı Count Vectors İle LogisticRegression algoritmasında oldu.